In [1]:
# %%
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import json, random, math
from typing import Iterable, Tuple, List, Dict

# Config
TOKENIZED_FILE = '../Assignment_1/tokenizer/tokenized_gu_tokens.jsonl'
VAL_SIZE = 1000
TEST_SIZE = 1000
SEED = 42
random.seed(SEED)


In [2]:
# %%
import json, random

MAX_SENTENCES = 1_000_000  # only first 10 lakh
LOG_EVERY = 10_000

def stream_first_sentences(path):
    """Yield up to 10 lakh non-empty sentences from the tokenized JSONL file."""
    count = 0
    with open(path, 'r', encoding='utf-8') as f:
        for idx, line in enumerate(f, start=1):
            try:
                obj = json.loads(line.strip())
                token_blocks = obj.get("tokens", [])
                if not token_blocks:
                    continue
                for block in token_blocks:
                    if block:  # non-empty sentence
                        yield block
                        count += 1
                        if count % LOG_EVERY == 0:
                            print(f"✅ Loaded {count:,} sentences so far...")
                        if count >= MAX_SENTENCES:
                            print(f"🛑 Reached limit of {MAX_SENTENCES:,} sentences.")
                            return
            except json.JSONDecodeError:
                continue

# Test quick load (will stop early)
sample_stream = stream_first_sentences(TOKENIZED_FILE)
sample = next(sample_stream)
print("Sample loaded sentence:", sample)


Sample loaded sentence: ['આ', 'વીડિયો', 'જુઓ', ':', 'ઊંઝા', 'માર્કેટયાર્ડ', 'આજથી', '25', 'જુલાઈ', 'સુધી', 'બંધ']


In [3]:
# %%
def stream_sentences(path: str) -> Iterable[List[str]]:
    """Stream tokenized sentences from a huge JSONL file efficiently."""
    with open(path, 'r', encoding='utf-8') as fh:
        for line in fh:
            line = line.strip()
            if not line:
                continue
            try:
                entry = json.loads(line)
                token_groups = entry.get('tokens', [])
                if not token_groups:
                    continue
                for sent in token_groups:
                    if sent and isinstance(sent, list) and len(sent) > 0:
                        yield sent
            except Exception:
                continue

# Test stream
sample_stream = list(next(stream_sentences(TOKENIZED_FILE)) for _ in range(3))
print("✅ Sample sentences from stream:", sample_stream)


✅ Sample sentences from stream: [['આ', 'વીડિયો', 'જુઓ', ':', 'ઊંઝા', 'માર્કેટયાર્ડ', 'આજથી', '25', 'જુલાઈ', 'સુધી', 'બંધ'], ['આ', 'વીડિયો', 'જુઓ', ':', 'ઊંઝા', 'માર્કેટયાર્ડ', 'આજથી', '25', 'જુલાઈ', 'સુધી', 'બંધ'], ['આ', 'વીડિયો', 'જુઓ', ':', 'ઊંઝા', 'માર્કેટયાર્ડ', 'આજથી', '25', 'જુલાઈ', 'સુધી', 'બંધ']]


In [4]:
# %%
def create_splits(path: str, val_size=1000, test_size=1000, seed=SEED, max_sentences=200_000, log_every=10_000):
    """
    Create train/val/test splits using streaming & reservoir sampling.
    - Reads up to `max_sentences`
    - Skips blank sentences
    - Prints progress every `log_every`
    """
    val, test, train = [], [], []
    random.seed(seed)
    i = 0

    for sent in stream_sentences(path):
        if not sent or len(sent) == 0:
            continue  # skip empty sentences

        i += 1
        if i % log_every == 0:
            print(f"⏳ Processed {i:,} sentences...")

        # Limit to first `max_sentences`
        if i > max_sentences:
            print(f"🛑 Reached {max_sentences:,} sentences limit — stopping stream.")
            break

        # Reservoir sampling for val/test, rest go to train
        if len(val) < val_size:
            val.append(sent)
        elif random.random() < val_size / i:
            val[random.randrange(val_size)] = sent
        elif len(test) < test_size:
            test.append(sent)
        elif random.random() < test_size / i:
            test[random.randrange(test_size)] = sent
        else:
            train.append(sent)

    print(f"\n✅ Completed streaming {i:,} sentences.")
    print(f"📊 Train: {len(train):,}, Val: {len(val):,}, Test: {len(test):,}")
    return train, val, test


# Run the split creation
train, val, test = create_splits(TOKENIZED_FILE)


⏳ Processed 10,000 sentences...
⏳ Processed 20,000 sentences...
⏳ Processed 30,000 sentences...
⏳ Processed 40,000 sentences...
⏳ Processed 50,000 sentences...
⏳ Processed 60,000 sentences...
⏳ Processed 70,000 sentences...
⏳ Processed 80,000 sentences...
⏳ Processed 90,000 sentences...
⏳ Processed 100,000 sentences...
⏳ Processed 110,000 sentences...
⏳ Processed 120,000 sentences...
⏳ Processed 130,000 sentences...
⏳ Processed 140,000 sentences...
⏳ Processed 150,000 sentences...
⏳ Processed 160,000 sentences...
⏳ Processed 170,000 sentences...
⏳ Processed 180,000 sentences...
⏳ Processed 190,000 sentences...
⏳ Processed 200,000 sentences...
🛑 Reached 200,000 sentences limit — stopping stream.

✅ Completed streaming 200,001 sentences.
📊 Train: 188,733, Val: 1,000, Test: 1,000


In [11]:
# %%
import json
import logging
from collections import Counter
from typing import List, Iterable, Any

# ================================
# CONFIG
# ================================
TOKENIZED_FILE = "../Assignment_1/tokenizer/tokenized_gu_tokens.jsonl"
MAX_SENTENCES = 200_000     # process only 2 lakh sentences
MAX_N = 4                   # up to 4-grams (quadrigrams)
LOG_EVERY = 10_000          # log every 10k sentences

# ================================
# LOGGING SETUP
# ================================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("ngram_build.log", mode="w", encoding="utf-8"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# ================================
# HELPERS
# ================================
def flatten_tokens(obj: Any, out: List[str]) -> None:
    """
    Flatten a nested list/tuple structure into `out`.
    Supports arbitrary nesting depth for safety (but typical data is shallow).
    """
    if obj is None:
        return
    if isinstance(obj, (str,)):
        out.append(obj)
    elif isinstance(obj, (list, tuple)):
        for item in obj:
            flatten_tokens(item, out)
    else:
        # ignore non-str, non-list items
        try:
            out.append(str(obj))
        except Exception:
            pass

def stream_first_sentences(file_path: str, max_sentences: int) -> Iterable[List[str]]:
    """
    Yield flattened token lists (one list per sentence) up to max_sentences.
    Handles records where 'tokens' field can be:
      - a flat list of tokens
      - a list of lists (multiple token lists)
      - nested lists (rare)
    Skips empty/malformed entries.
    """
    yielded = 0
    with open(file_path, 'r', encoding='utf-8') as fh:
        for lineno, line in enumerate(fh, start=1):
            if yielded >= max_sentences:
                break
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except Exception:
                # skip malformed JSON line
                if lineno % 100_000 == 0:
                    logger.warning(f"Skipped malformed JSON at line {lineno}")
                continue

            # Expected shape: {'tokens': [...] } or sometimes a raw list
            tokens_field = None
            if isinstance(obj, dict) and 'tokens' in obj:
                tokens_field = obj['tokens']
            elif isinstance(obj, list):
                tokens_field = obj
            else:
                # skip unknown record shapes
                continue

            # Flatten token structure into a single list of strings
            flat = []
            flatten_tokens(tokens_field, flat)

            # skip empty after flattening
            if not flat:
                continue

            yielded += 1
            if yielded % LOG_EVERY == 0:
                logger.info(f"📖 Loaded {yielded:,}/{max_sentences:,} sentences ({yielded/max_sentences*100:.2f}%)")
            yield flat

    logger.info(f"✅ Finished streaming (yielded {yielded:,} sentences).")

def ngrams_from_sentence(tokens: List[str], n: int):
    """Yield n-gram tuples from a token list."""
    if not tokens or n <= 0:
        return
    L = len(tokens)
    for i in range(L - n + 1):
        # slice returns list -> wrap as tuple so it's hashable
        yield tuple(tokens[i:i+n])

def build_ngram_counts(sent_stream: Iterable[List[str]], max_n: int = 4, log_every: int = 10_000):
    """
    Build n-gram counts for n in [1..max_n].
    Returns (counters, total_unigrams, vocab_set)
    """
    counters = {n: Counter() for n in range(1, max_n + 1)}
    vocab = set()
    total_unigrams = 0
    processed = 0

    for processed, sent in enumerate(sent_stream, start=1):
        # update vocab and unigram total
        for t in sent:
            vocab.add(t)
        total_unigrams += len(sent)

        # update n-gram counters
        for n in range(1, max_n + 1):
            for ng in ngrams_from_sentence(sent, n):
                counters[n][ng] += 1

        if processed % log_every == 0:
            logger.info(f"✅ Processed {processed:,}/{MAX_SENTENCES:,} sentences ({processed/MAX_SENTENCES*100:.2f}%)")

    logger.info(f"🎯 N-gram counting complete after {processed:,} sentences.")
    return counters, total_unigrams, vocab

# ================================
# RUN: build counts
# ================================
logger.info(f"🔨 Building up to {MAX_N}-gram counts for {MAX_SENTENCES:,} sentences...\n")
sentence_stream = stream_first_sentences(TOKENIZED_FILE, MAX_SENTENCES)
counts, total_unigrams, vocab = build_ngram_counts(sentence_stream, max_n=MAX_N, log_every=LOG_EVERY)

# Summary
logger.info(f"🧩 1-gram unique count: {len(counts[1]):,}")
logger.info(f"🧩 2-gram unique count: {len(counts[2]):,}")
logger.info(f"🧩 3-gram unique count: {len(counts[3]):,}")
logger.info(f"🧩 4-gram unique count: {len(counts[4]):,}")
logger.info(f"📊 Total unigram tokens: {total_unigrams:,}")
logger.info(f"📚 Vocabulary size: {len(vocab):,}")


2025-10-29 23:23:30,959 [INFO] 🔨 Building up to 4-gram counts for 200,000 sentences...

2025-10-29 23:23:36,899 [INFO] 📖 Loaded 10,000/200,000 sentences (5.00%)
2025-10-29 23:23:36,902 [INFO] ✅ Processed 10,000/200,000 sentences (5.00%)
2025-10-29 23:23:41,613 [INFO] 📖 Loaded 20,000/200,000 sentences (10.00%)
2025-10-29 23:23:41,621 [INFO] ✅ Processed 20,000/200,000 sentences (10.00%)
2025-10-29 23:23:46,187 [INFO] 📖 Loaded 30,000/200,000 sentences (15.00%)
2025-10-29 23:23:46,277 [INFO] ✅ Processed 30,000/200,000 sentences (15.00%)
2025-10-29 23:23:51,233 [INFO] 📖 Loaded 40,000/200,000 sentences (20.00%)
2025-10-29 23:23:51,238 [INFO] ✅ Processed 40,000/200,000 sentences (20.00%)
2025-10-29 23:23:55,480 [INFO] 📖 Loaded 50,000/200,000 sentences (25.00%)
2025-10-29 23:23:55,484 [INFO] ✅ Processed 50,000/200,000 sentences (25.00%)
2025-10-29 23:24:00,045 [INFO] 📖 Loaded 60,000/200,000 sentences (30.00%)
2025-10-29 23:24:00,048 [INFO] ✅ Processed 60,000/200,000 sentences (30.00%)
2025-10-

In [12]:
# %%
import math
import logging
from collections import defaultdict, Counter

logger = logging.getLogger(__name__)

# ---- Config for generation ----
CANDIDATE_VOCAB_SIZE = 5000   # use top 5k unigrams as candidates during generation (pruning)
BEAM_SIZE = 20
MAX_GEN_LEN = 20

# ---- Quick sanity-checks ----
assert 1 in counts and 2 in counts and 3 in counts and 4 in counts, "counts[1..4] required"

# ---- Totals ----
total_unigram_tokens = total_unigrams  # from earlier
vocab_size = len(vocab)
logger.info(f"Using vocabulary size = {vocab_size}, total unigrams = {total_unigram_tokens:,}")

# ---- Precompute totals for each n ----
counts_total = {n: sum(counts[n].values()) for n in range(1,5)}
logger.info("Total counts per order: " + ", ".join(f"{n}:{counts_total[n]:,}" for n in counts_total))

# ---- Precompute continuation counts needed for Kneser-Ney ----
# continuation_unigram[w] = number of unique bigram contexts (*, w)
continuation_unigram = Counter()
for (w1, w2), c in counts[2].items():
    continuation_unigram[w2] += 1

total_bigram_types = len(counts[2])
logger.info(f"Total bigram types: {total_bigram_types:,}")

# For trigram and 4-gram continuation computations, we need:
#   N1+(history) = number of unique words that follow that history
unique_continuations = {n: defaultdict(set) for n in (1,2,3)}  # history -> set of continuations
# For bigram history (w2) we want unique words that follow w2 in (w2,w3) ... etc.

# Build for histories of length 1 (for bigram histories), 2, 3
for (w1, w2), _ in counts[2].items():
    unique_continuations[1][(w2,)].add(w1)  # not used directly but safe

for (w1, w2, w3), _ in counts[3].items():
    hist2 = (w1, w2)
    unique_continuations[2][hist2].add(w3)

for (w1, w2, w3, w4), _ in counts[4].items():
    hist3 = (w1, w2, w3)
    unique_continuations[3][hist3].add(w4)

# Convert sets -> counts (N+)
Nplus = {1: {}, 2: {}, 3: {}}
for n in (1,2,3):
    for h, s in unique_continuations[n].items():
        Nplus[n][h] = len(s)

logger.info("Precomputations done: continuation_unigram (len)=%d, Nplus keys: %d/%d/%d",
            len(continuation_unigram), len(Nplus[1]), len(Nplus[2]), len(Nplus[3]))

# ---- Top candidate vocab for generation (pruning) based on unigram frequency ----
top_unigrams = [w for (w,), _ in counts[1].most_common(CANDIDATE_VOCAB_SIZE)]
logger.info("Candidate vocabulary size for generation: %d", len(top_unigrams))


2025-10-29 23:43:45,530 [INFO] Using vocabulary size = 393430, total unigrams = 9,475,762
2025-10-29 23:43:46,773 [INFO] Total counts per order: 1:9,475,762, 2:9,275,762, 3:9,075,901, 4:8,878,254
2025-10-29 23:43:59,754 [INFO] Total bigram types: 3,562,996
2025-10-29 23:55:22,636 [INFO] Precomputations done: continuation_unigram (len)=384580, Nplus keys: 384580/3534602/6425960
2025-10-29 23:55:23,949 [INFO] Candidate vocabulary size for generation: 5000


In [13]:
# %%
from functools import lru_cache

# Discount value used in modified Kneser-Ney
D = 0.75

# Precompute sums for lower orders used by KN
total_bigram_types = len(counts[2])

# Kneser-Ney unigram continuation prob
@lru_cache(maxsize=500000)
def kn_unigram_continuation_prob(w):
    # P_continuation(w) = #contexts that w appears in (as last in bigram) / total number of bigram types
    return continuation_unigram.get(w, 0) / total_bigram_types if total_bigram_types > 0 else 0.0

# recursive KN: P(w4 | w1 w2 w3)
@lru_cache(maxsize=500000)
def kn_prob_4(w1, w2, w3, w4):
    # counts
    hist3 = (w1, w2, w3)
    c3 = counts[3].get(hist3, 0)
    c4 = counts[4].get((w1,w2,w3,w4), 0)

    # If history count > 0 use formula; else back off
    if c3 > 0:
        # first term
        term1 = max(c4 - D, 0) / c3

        # lambda = D * (number of unique continuations of hist3) / c3
        N1plus = Nplus[3].get(hist3, 0)
        lamb = (D * N1plus) / c3 if c3>0 else 0.0

        # lower-order (3-gram) probability P_kn(w4 | w2 w3)
        lower = kn_prob_3(w2, w3, w4)
        return term1 + lamb * lower
    else:
        # backoff to 3-gram
        return kn_prob_3(w2, w3, w4)

@lru_cache(maxsize=500000)
def kn_prob_3(w1, w2, w3):
    hist2 = (w1, w2)
    c2 = counts[2].get(hist2, 0)
    c3 = counts[3].get((w1,w2,w3), 0)
    if c2 > 0:
        term1 = max(c3 - D, 0) / c2
        N1plus = Nplus[2].get(hist2, 0)
        lamb = (D * N1plus) / c2
        lower = kn_prob_2(w2, w3)
        return term1 + lamb * lower
    else:
        return kn_prob_2(w2, w3)

@lru_cache(maxsize=500000)
def kn_prob_2(w1, w2):
    hist1 = (w1,)
    c1 = counts[1].get(hist1, 0)  # note: counts[1] keys are (w,), so hist1=(w1,)
    c2 = counts[2].get((w1,w2), 0)
    if c1 > 0:
        term1 = max(c2 - D, 0) / c1
        N1plus = Nplus[1].get(hist1, 0)
        lamb = (D * N1plus) / c1 if c1>0 else 0.0
        lower = kn_unigram_continuation_prob(w2)  # final lower
        return term1 + lamb * lower
    else:
        return kn_unigram_continuation_prob(w2)

logger.info("Modified Kneser-Ney functions ready.")


2025-10-30 00:00:02,009 [INFO] Modified Kneser-Ney functions ready.


In [14]:
# %%
from functools import lru_cache

# Katz discount value
KATZ_D = 0.5

# Precompute lower-order MLE (add-one smoothed) probabilities for normalization
unigram_total = counts_total[1]
def mle_unigram_prob(w):
    # add-one smoothing
    return (counts[1].get((w,), 0) + 1) / (unigram_total + vocab_size)

@lru_cache(maxsize=500000)
def katz_prob_4(w1,w2,w3,w4):
    """
    Approximate Katz backoff probability for quadrigram:
      - If seen: (c-d)/c(hist)
      - Else: backoff weight * Katz prob of lower order (trigram)
    We compute backoff weight alpha(hist) from leftover mass and lower-order sum.
    """
    quad = (w1,w2,w3,w4)
    tri_hist = (w1,w2,w3)
    c_tri = counts[3].get(tri_hist, 0)
    c_quad = counts[4].get(quad, 0)

    if c_tri > 0 and c_quad > 0:
        return max(c_quad - KATZ_D, 0) / c_tri

    # compute alpha for tri_hist
    # sum_seen = sum((c - d)/c_tri for each seen continuation)
    seen_conts = [w4_ for (a,b,c,d), cval in counts[4].items() if (a,b,c) == tri_hist]
    if c_tri == 0:
        # backoff to trigram MLE
        return katz_prob_3(w2,w3,w4)
    sum_seen = 0.0
    sum_lower_seen = 0.0
    for cont in seen_conts:
        c_q = counts[4].get((tri_hist[0],tri_hist[1],tri_hist[2],cont),0)
        sum_seen += max(c_q - KATZ_D, 0) / c_tri
        # accumulate lower-order prob for seen cont
        sum_lower_seen += katz_prob_3(tri_hist[1], tri_hist[2], cont)

    leftover = max(1.0 - sum_seen, 0.0)
    # lower_total = sum over all vocab of lower_prob; should be 1.0
    lower_total = 1.0
    lower_unseen_mass = max(lower_total - sum_lower_seen, 1e-12)
    alpha = leftover / lower_unseen_mass
    return alpha * katz_prob_3(w2,w3,w4)

@lru_cache(maxsize=500000)
def katz_prob_3(w1,w2,w3):
    tri = (w1,w2,w3)
    bi_hist = (w1,w2)
    c_bi = counts[2].get(bi_hist, 0)
    c_tri = counts[3].get(tri, 0)
    if c_bi > 0 and c_tri > 0:
        return max(c_tri - KATZ_D, 0) / c_bi
    # else backoff:
    # compute alpha for bi_hist similarly
    seen_conts = [t3 for (a,b,c), cval in counts[3].items() if (a,b)==bi_hist]
    if c_bi == 0:
        return mle_unigram_prob(w3)
    sum_seen = 0.0
    sum_lower_seen = 0.0
    for cont in seen_conts:
        c_t = counts[3].get((bi_hist[0],bi_hist[1],cont),0)
        sum_seen += max(c_t - KATZ_D, 0) / c_bi
        sum_lower_seen += mle_unigram_prob(cont)
    leftover = max(1.0 - sum_seen, 0.0)
    lower_unseen_mass = max(1.0 - sum_lower_seen, 1e-12)
    alpha = leftover / lower_unseen_mass
    return alpha * mle_unigram_prob(w3)

logger.info("Katz Backoff functions (approximate) ready.")


2025-10-30 00:00:23,690 [INFO] Katz Backoff functions (approximate) ready.


In [15]:
# %%
import random
import numpy as np

def generate_greedy(start_tokens=None, max_len=MAX_GEN_LEN, model='kneser'):
    if start_tokens is None:
        # start with three most frequent unigrams (or less)
        start_tokens = [w for (w,), _ in counts[1].most_common(3)]
    sent = list(start_tokens[:3])  # ensure at least 3 tokens
    logger.info("Greedy generation start: " + " ".join(sent))

    for step in range(max_len - len(sent)):
        best_word = None
        best_score = -float('inf')
        # candidate pruning
        candidates = top_unigrams  # list of words (strings)
        for w in candidates:
            if model == 'kneser':
                p = kn_prob_4(sent[-3], sent[-2], sent[-1], w)
            elif model == 'katz':
                p = katz_prob_4(sent[-3], sent[-2], sent[-1], w)
            else:
                # default to KN
                p = kn_prob_4(sent[-3], sent[-2], sent[-1], w)
            # use log-score
            if p <= 0:
                score = -1e300
            else:
                score = math.log(p)
            if score > best_score:
                best_score = score
                best_word = w
        sent.append(best_word)
        if (step+1) % 5 == 0:
            logger.info(f"Greedy generated ({len(sent)}): {' '.join(sent[:10])} ...")
        if best_word in ['.', '।']:
            break
    return " ".join(sent)

# Example
logger.info("Generating 3 greedy sentences (Kneser-Ney):")
for i in range(3):
    print("GREEDY:", generate_greedy(model='kneser'))


2025-10-30 00:00:53,305 [INFO] Generating 3 greedy sentences (Kneser-Ney):
2025-10-30 00:00:53,435 [INFO] Greedy generation start: . છે ,
2025-10-30 00:01:33,349 [INFO] Greedy generated (8): . છે , અને તે પણ એક મોટો ...
2025-10-30 00:01:37,865 [INFO] Greedy generation start: . છે ,
2025-10-30 00:01:37,902 [INFO] Greedy generated (8): . છે , અને તે પણ એક મોટો ...


GREEDY: . છે , અને તે પણ એક મોટો સવાલ છે .
GREEDY: . છે , અને તે પણ એક મોટો સવાલ છે .


2025-10-30 00:01:38,043 [INFO] Greedy generation start: . છે ,
2025-10-30 00:01:38,097 [INFO] Greedy generated (8): . છે , અને તે પણ એક મોટો ...


GREEDY: . છે , અને તે પણ એક મોટો સવાલ છે .


In [16]:
# %%
import heapq

def generate_beam(start_tokens=None, beam_size=BEAM_SIZE, max_len=MAX_GEN_LEN, model='kneser'):
    if start_tokens is None:
        start_tokens = [w for (w,), _ in counts[1].most_common(3)]
    initial = list(start_tokens[:3])
    # beams: list of tuples (score, seq)
    beams = [(0.0, initial)]
    for step in range(max_len - len(initial)):
        new_beams = []
        for score, seq in beams:
            # candidate pruning
            for w in top_unigrams:
                if model == 'kneser':
                    p = kn_prob_4(seq[-3], seq[-2], seq[-1], w)
                elif model == 'katz':
                    p = katz_prob_4(seq[-3], seq[-2], seq[-1], w)
                else:
                    p = kn_prob_4(seq[-3], seq[-2], seq[-1], w)
                if p <= 0:
                    continue
                new_seq = seq + [w]
                new_score = score + math.log(p)
                new_beams.append((new_score, new_seq))
        if not new_beams:
            break
        # keep top beam_size by score (higher is better)
        new_beams.sort(key=lambda x: x[0], reverse=True)
        beams = new_beams[:beam_size]
        if step % 2 == 0:
            logger.info(f"Beam step {step}: top score {beams[0][0]:.2f}, seq head: {' '.join(beams[0][1][:8])}...")
        # stop early if best beam ends with sentence-end token
        if beams and beams[0][1][-1] in ['.', '।']:
            break
    best = max(beams, key=lambda x: x[0])
    return " ".join(best[1])

# Example
logger.info("Generating 2 beam-search sentences (beam=20, Kneser-Ney):")
for i in range(2):
    print("BEAM:", generate_beam(model='kneser', beam_size=20))


2025-10-30 00:10:12,178 [INFO] Generating 2 beam-search sentences (beam=20, Kneser-Ney):
2025-10-30 00:10:12,957 [INFO] Beam step 0: top score -1.59, seq head: . છે , અને...
2025-10-30 00:10:29,341 [INFO] Beam step 2: top score -5.38, seq head: . છે , તો પછી તમે...
2025-10-30 00:10:32,896 [INFO] Beam step 4: top score -7.36, seq head: . છે , અને તે જ સમયે ,...
2025-10-30 00:10:36,440 [INFO] Beam step 6: top score -8.90, seq head: . છે , છોડ પરના પાંદડા પાતળા હોય...
2025-10-30 00:10:40,489 [INFO] Beam step 8: top score -12.58, seq head: . છે , છોડ પરના પાંદડા પાતળા હોય...
2025-10-30 00:10:42,540 [INFO] Beam step 0: top score -1.59, seq head: . છે , અને...


BEAM: . છે , અને તેથી પર . ડી . સી . બી .


2025-10-30 00:19:22,380 [INFO] Beam step 2: top score -5.38, seq head: . છે , તો પછી તમે...
2025-10-30 00:19:44,242 [INFO] Beam step 4: top score -7.36, seq head: . છે , અને તે જ સમયે ,...
2025-10-30 00:19:50,826 [INFO] Beam step 6: top score -8.90, seq head: . છે , છોડ પરના પાંદડા પાતળા હોય...
2025-10-30 00:19:59,974 [INFO] Beam step 8: top score -12.58, seq head: . છે , છોડ પરના પાંદડા પાતળા હોય...


BEAM: . છે , અને તેથી પર . ડી . સી . બી .
